<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Hannah", "transactions": [{"transaction-id": 4274, "amount": -835}]}\n',)

In [5]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [6]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [7]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [8]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Hannah", "transactions": [{"transaction-id": 4274, "amount": -835}]}\n',
 '{"id": 1, "name": "Patricia", "transactions": [{"transaction-id": 742, "amount": 1782}, {"transaction-id": 853, "amount": 1862}, {"transaction-id": 1720, "amount": 1892}, {"transaction-id": 1776, "amount": 1961}, {"transaction-id": 1890, "amount": 1851}, {"transaction-id": 2230, "amount": 1942}, {"transaction-id": 2820, "amount": 1807}, {"transaction-id": 3141, "amount": 1816}, {"transaction-id": 3605, "amount": 1922}, {"transaction-id": 4442, "amount": 1889}, {"transaction-id": 4491, "amount": 2001}, {"transaction-id": 4506, "amount": 1791}, {"transaction-id": 4690, "amount": 1833}, {"transaction-id": 4889, "amount": 1752}, {"transaction-id": 6177, "amount": 1841}, {"transaction-id": 6296, "amount": 1896}, {"transaction-id": 6513, "amount": 1798}, {"transaction-id": 6588, "amount": 1822}, {"transaction-id": 6944, "amount": 1882}, {"transaction-id": 7081, "amount": 1715}, {"transaction-id": 

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [9]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Hannah',
  'transactions': [{'transaction-id': 4274, 'amount': -835}]},
 {'id': 1,
  'name': 'Patricia',
  'transactions': [{'transaction-id': 742, 'amount': 1782},
   {'transaction-id': 853, 'amount': 1862},
   {'transaction-id': 1720, 'amount': 1892},
   {'transaction-id': 1776, 'amount': 1961},
   {'transaction-id': 1890, 'amount': 1851},
   {'transaction-id': 2230, 'amount': 1942},
   {'transaction-id': 2820, 'amount': 1807},
   {'transaction-id': 3141, 'amount': 1816},
   {'transaction-id': 3605, 'amount': 1922},
   {'transaction-id': 4442, 'amount': 1889},
   {'transaction-id': 4491, 'amount': 2001},
   {'transaction-id': 4506, 'amount': 1791},
   {'transaction-id': 4690, 'amount': 1833},
   {'transaction-id': 4889, 'amount': 1752},
   {'transaction-id': 6177, 'amount': 1841},
   {'transaction-id': 6296, 'amount': 1896},
   {'transaction-id': 6513, 'amount': 1798},
   {'transaction-id': 6588, 'amount': 1822},
   {'transaction-id': 6944, 'amount': 1882},
   {

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [10]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 5,
  'name': 'Alice',
  'transactions': [{'transaction-id': 12, 'amount': 2452},
   {'transaction-id': 16, 'amount': 1911},
   {'transaction-id': 25, 'amount': 2490},
   {'transaction-id': 230, 'amount': 1815},
   {'transaction-id': 288, 'amount': 1888},
   {'transaction-id': 349, 'amount': 2119},
   {'transaction-id': 409, 'amount': 2419},
   {'transaction-id': 415, 'amount': 1752},
   {'transaction-id': 451, 'amount': 2097},
   {'transaction-id': 489, 'amount': 2374},
   {'transaction-id': 517, 'amount': 2164},
   {'transaction-id': 658, 'amount': 2333},
   {'transaction-id': 783, 'amount': 2108},
   {'transaction-id': 851, 'amount': 2118},
   {'transaction-id': 927, 'amount': 2005},
   {'transaction-id': 986, 'amount': 2011},
   {'transaction-id': 1092, 'amount': 2043},
   {'transaction-id': 1099, 'amount': 2101},
   {'transaction-id': 1254, 'amount': 2423},
   {'transaction-id': 1256, 'amount': 2201},
   {'transaction-id': 1373, 'amount': 2357},
   {'transaction-id': 1444, 

In [11]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 160},
 {'name': 'Alice', 'count': 16},
 {'name': 'Alice', 'count': 47},
 {'name': 'Alice', 'count': 77},
 {'name': 'Alice', 'count': 17})

In [12]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(160, 16, 47, 77, 17)

In [13]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

46.87777777777778

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [14]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 12, 'amount': 2452},
  {'transaction-id': 16, 'amount': 1911},
  {'transaction-id': 25, 'amount': 2490},
  {'transaction-id': 230, 'amount': 1815},
  {'transaction-id': 288, 'amount': 1888},
  {'transaction-id': 349, 'amount': 2119},
  {'transaction-id': 409, 'amount': 2419},
  {'transaction-id': 415, 'amount': 1752},
  {'transaction-id': 451, 'amount': 2097},
  {'transaction-id': 489, 'amount': 2374},
  {'transaction-id': 517, 'amount': 2164},
  {'transaction-id': 658, 'amount': 2333},
  {'transaction-id': 783, 'amount': 2108},
  {'transaction-id': 851, 'amount': 2118},
  {'transaction-id': 927, 'amount': 2005},
  {'transaction-id': 986, 'amount': 2011},
  {'transaction-id': 1092, 'amount': 2043},
  {'transaction-id': 1099, 'amount': 2101},
  {'transaction-id': 1254, 'amount': 2423},
  {'transaction-id': 1256, 'amount': 2201},
  {'transaction-id': 1373, 'amount': 2357},
  {'transaction-id': 1444, 'amount': 2130},
  {'transaction-id': 1512, 'amount': 1996},
  {'tra

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 12, 'amount': 2452},
 {'transaction-id': 16, 'amount': 1911},
 {'transaction-id': 25, 'amount': 2490})

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(2452, 1911, 2490)

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1869.9974401516947

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [18]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [19]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [20]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [21]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [22]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 99), ('Alice', 104), ('Alice', 117), ('Alice', 130), ('Bob', 33), ('Bob', 36), ('Bob', 39), ('Bob', 42), ('Charlie', 110), ('Charlie', 120), ('Charlie', 130), ('Charlie', 140), ('Dan', 55), ('Dan', 60), ('Dan', 65), ('Dan', 69), ('Edith', 117), ('Edith', 127), ('Edith', 136), ('Edith', 148), ('Frank', 147), ('Frank', 158), ('Frank', 175), ('Frank', 192), ('George', 110), ('George', 120), ('George', 130), ('George', 140), ('Hannah', 89), ('Hannah', 100), ('Hannah', 107), ('Hannah', 116), ('Ingrid', 154), ('Ingrid', 162), ('Ingrid', 177), ('Ingrid', 192), ('Jerry', 87), ('Jerry', 92), ('Jerry', 103), ('Jerry', 112), ('Kevin', 77), ('Kevin', 84), ('Kevin', 91), ('Kevin', 98), ('Laura', 99), ('Laura', 110), ('Laura', 177), ('Michael', 99), ('Michael', 108), ('Michael', 117), ('Michael', 126), ('Norbert', 75), ('Norbert', 80), ('Norbert', 87), ('Norbert', 95), ('Oliver', 143), ('Oliver', 156), ('Oliver', 169), ('Oliver', 181), ('Patricia', 55), ('Patricia', 60), ('Patricia', 65),

In [23]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 450), ('Bob', 150), ('Charlie', 500), ('Dan', 249), ('Edith', 528), ('Frank', 672), ('George', 500), ('Hannah', 412), ('Ingrid', 685), ('Jerry', 394), ('Kevin', 350), ('Laura', 386), ('Michael', 450), ('Norbert', 337), ('Oliver', 649), ('Patricia', 250), ('Quinn', 298), ('Ray', 650), ('Sarah', 797), ('Tim', 350), ('Ursula', 499), ('Victor', 748), ('Wendy', 585), ('Xavier', 449), ('Yvonne', 250), ('Zelda', 631)]
CPU times: user 175 ms, sys: 5.45 ms, total: 180 ms
Wall time: 621 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [24]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [25]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Hannah,"[{'transaction-id': 4274, 'amount': -835}]"
1,1,Patricia,"[{'transaction-id': 742, 'amount': 1782}, {'tr..."
2,2,Edith,"[{'transaction-id': 62, 'amount': -2819}, {'tr..."
3,3,Ingrid,"[{'transaction-id': 118, 'amount': 4012}, {'tr..."
4,4,Michael,"[{'transaction-id': 18, 'amount': 820}, {'tran..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [26]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 204 ms, sys: 29.4 ms, total: 233 ms
Wall time: 1.67 s


name
Alice      450
Bob        150
Charlie    500
Dan        249
Edith      528
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [27]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Hannah', 'amount': -835, 'transaction-id': 4274},
 {'id': 1, 'name': 'Patricia', 'amount': 1782, 'transaction-id': 742},
 {'id': 1, 'name': 'Patricia', 'amount': 1862, 'transaction-id': 853})

In [28]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Hannah,-835,4274
1,1,Patricia,1782,742
2,1,Patricia,1862,853
3,1,Patricia,1892,1720
4,1,Patricia,1961,1776


In [29]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 226 ms, sys: 6.28 ms, total: 232 ms
Wall time: 1.45 s


name
Alice       21095
Bob          3025
Charlie     24154
Dan         14339
Edith       30323
Frank       20732
George      35135
Hannah       5820
Ingrid      20538
Jerry       15659
Kevin       14379
Laura       12144
Michael     21572
Norbert      6772
Oliver      27233
Patricia     9870
Quinn        5292
Ray         30138
Sarah       42074
Tim          7462
Ursula      22444
Victor      25996
Wendy       27532
Xavier      16359
Yvonne       5106
Zelda       34807
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [30]:
client.shutdown()